In [2]:
from nustar_pysolar import planning, io
import astropy.units as u

# Download the list of occultation periods from the MOC at Berkeley.

## Note that the occultation periods typically only are stored at Berkeley for the *future* and not for the past. So this is only really useful for observation planning.

In [3]:
fname = io.download_occultation_times(outdir='../data/')
print(fname)

../data/NUSTAR.2018_148.SHADOW_ANALYSIS.txt


# Download the NuSTAR TLE archive.

This contains every two-line element (TLE) that we've received for the whole mission. We'll expand on how to use this later.

The `times`, `line1`, and `line2` elements are now the TLE elements for each epoch.

In [4]:
tlefile = io.download_tle(outdir='../data')
print(tlefile)
times, line1, line2 = io.read_tle_file(tlefile)

../data/NuSTAR.tle


# Here is where we define the observing window that we want to use.

Note that tstart and tend must be in the future otherwise you won't find any occultation times and sunlight_periods will return an error.

In [5]:
tstart = '2018-05-28T15:00:00'
tend = '2018-05-29T12:00:00'
orbits = planning.sunlight_periods(fname, tstart, tend)

# We want to know how to orient NuSTAR for the Sun.

We can more or less pick any angle that we want. But this angle has to be specified a little in advance so that the NuSTAR SOC can plan the "slew in" maneuvers. Below puts DET0 in the top left corner (north-east with respect to RA/Dec coordinates).

### This is what you tell the SOC you want the "Sky PA angle" to be.

In [8]:
orbits

[[datetime.datetime(2018, 5, 28, 14, 8, 10),
  datetime.datetime(2018, 5, 28, 15, 10)],
 [datetime.datetime(2018, 5, 28, 15, 44, 50),
  datetime.datetime(2018, 5, 28, 16, 46, 40)],
 [datetime.datetime(2018, 5, 28, 17, 21, 30),
  datetime.datetime(2018, 5, 28, 18, 23, 20)],
 [datetime.datetime(2018, 5, 28, 18, 58, 10),
  datetime.datetime(2018, 5, 28, 20, 0)],
 [datetime.datetime(2018, 5, 28, 20, 34, 50),
  datetime.datetime(2018, 5, 28, 21, 36, 40)],
 [datetime.datetime(2018, 5, 28, 22, 11, 30),
  datetime.datetime(2018, 5, 28, 23, 13, 10)],
 [datetime.datetime(2018, 5, 28, 23, 48, 10),
  datetime.datetime(2018, 5, 29, 0, 49, 50)],
 [datetime.datetime(2018, 5, 29, 1, 24, 50),
  datetime.datetime(2018, 5, 29, 2, 26, 30)],
 [datetime.datetime(2018, 5, 29, 3, 1, 30),
  datetime.datetime(2018, 5, 29, 4, 3, 10)],
 [datetime.datetime(2018, 5, 29, 4, 38, 10),
  datetime.datetime(2018, 5, 29, 5, 39, 50)],
 [datetime.datetime(2018, 5, 29, 6, 14, 50),
  datetime.datetime(2018, 5, 29, 7, 16, 30)]

In [5]:
pa = planning.get_nustar_roll(tstart, 0)
print("NuSTAR Roll angle for Det0 in NE quadrant: {}".format(pa))

NuSTAR Roll angle for Det0 in NE quadrant: 348.3764402915343 deg


# Set up the offset you want to use here:

The first element is the direction +WEST of the center of the Sun, the second is the offset +NORTH of the center of the Sun.

If you want multiple pointing locations you can either specify an array of offsets or do this "by hand" below.

In [6]:
offset = [-190., -47.]*u.arcsec

# Loop over each orbit and correct the pointing for the same heliocentric pointing position.

Note that you may want to update the pointing for solar rotation. That's up to the user to decide and is not done here.

In [7]:
for ind, orbit in enumerate(orbits):
    midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
    sky_pos = planning.get_sky_position(midTime, offset)
    print("Orbit: {}".format(ind))
    print("Orbit start: {} Orbit end: {}".format(orbit[0].isoformat(), orbit[1].isoformat()))
    print('Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), sky_pos[0], sky_pos[1]))
    print("")

Orbit: 0
Orbit start: 2017-06-10T11:25:40 Orbit end: 2017-06-10T12:27:30
Aim time: 2017-06-10T11:56:35 RA (deg): 78.69718876009287 deg Dec (deg): 23.0238274997264 deg

Orbit: 1
Orbit start: 2017-06-10T13:02:20 Orbit end: 2017-06-10T14:04:10
Aim time: 2017-06-10T13:33:15 RA (deg): 78.76668616118683 deg Dec (deg): 23.02881188444142 deg

Orbit: 2
Orbit start: 2017-06-10T14:39:00 Orbit end: 2017-06-10T15:40:50
Aim time: 2017-06-10T15:09:55 RA (deg): 78.83618771028821 deg Dec (deg): 23.033765804969477 deg

Orbit: 3
Orbit start: 2017-06-10T16:15:40 Orbit end: 2017-06-10T17:17:30
Aim time: 2017-06-10T16:46:35 RA (deg): 78.90569338454985 deg Dec (deg): 23.03868925382892 deg

Orbit: 4
Orbit start: 2017-06-10T17:52:20 Orbit end: 2017-06-10T18:54:20
Aim time: 2017-06-10T18:23:20 RA (deg): 78.97526308507165 deg Dec (deg): 23.04358642851394 deg



# Sanity check: Reproduce the 2016-07-26 pointing.

Based on the IDL version of this code, when the aim time was 2016-07-26 19:53:15.00 the NP was 8.86 degrees and the RA and Dec were 126.04630 and 19.33532 deg, respectively for an offset of +1000 arcseconds West and +150 arcseconds North of the center of the Sun.

In [8]:
aim_time = '2016-07-26T19:53:15.00'
offset = [1000, 150]*u.arcsec
sky_pos = planning.get_sky_position(aim_time, offset)
print(sky_pos)

[ 126.04053869   19.33666449] deg


In [9]:
np = planning.get_nustar_roll(aim_time, 0)
print(np)

8.865884432089102 deg
